In [1]:
def houghTransform(img) :
    #Hough Transform to detect the iris
    used_img = img

    #b,g,r = cv2.split(used_img)
    #rgb_img = cv2.merge([r,g,b])

    preprocessing = cv2.cvtColor(used_img, cv2.COLOR_BGR2GRAY)
    preprocessing = cv2.medianBlur(preprocessing, 5)

    param1 = 90
    #param2 = 550
    param2 = 400
    maxRadius = 0
    minRadius = 0

    rows = preprocessing.shape[0]
    circles = cv2.HoughCircles(preprocessing, cv2.HOUGH_GRADIENT, 7, 20, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    radius = 0
    img_center = (int(used_img.shape[0]/2),int(used_img.shape[1] /2))

    if circles is not None :
        circles = np.uint16(np.around(circles))
        beginning = True
        for i in circles[0, :]:
            if beginning == True :
                center = (i[0], i[1])
                radius = i[2]
                beginning = False
            else :
                old_min = abs(radius*2 - used_img.shape[0])
                new_min = abs(i[2]*2 - used_img.shape[0])
                #print(str(old_min) + " : " + str(new_min) )
                if old_min > new_min :
                    #drawing circle
                    center = (i[0], i[1])
                    radius = i[2]

            cv2.circle(preprocessing, center, radius, (0,255,0),2)
            #drawing center
            cv2.circle(preprocessing, center,2,(0,0,255),3)
    else :
        return "Circle not detected"
    
    new_radius = int(radius * 0.7)
    image = used_img
    height = int(radius - new_radius)
    width = int(radius * 2 * np.pi)
    r_in = new_radius
    r_out = radius

    thetas = np.arange(0, 2 * np.pi, 2 * np.pi / width)  # Theta values
    #r_out = r_in + r_out
    # Create empty flatten image
    flat = np.zeros((height,width, 3), np.uint8)
    circle_x = int(image.shape[0] / 2)
    circle_y = int(image.shape[1] / 2)

    #print(r_in)
    #print(r_out)
    #print(width)
    #print(height)
    for i in range(width):
        for j in range(height):
            theta = thetas[i]  # value of theta coordinate
            r_pro = j / height  # value of r coordinate(normalized)

            # get coordinate of boundaries
            Xi = circle_x + r_in * np.cos(theta)
            Yi = circle_y + r_in * np.sin(theta)

            Xo = circle_x + r_out * np.cos(theta)
            Yo = circle_y + r_out * np.sin(theta)

            # the matched cartesian coordinates for the polar coordinates
            Xc = (1 - r_pro) * Xi + r_pro * Xo
            Yc = (1 - r_pro) * Yi + r_pro * Yo

            if Xc >= image.shape[0] :
                Xc = image.shape[1] - 1

            if Yc >= image.shape[1] :
                Yc = image.shape[1] - 1
            #print(i)
            #print(j)
            color = image[int(Xc)][int(Yc)]  # color of the pixel
            flat[j][i] = color
    
    gray_flat = cv2.cvtColor(flat, cv2.COLOR_BGR2GRAY)

    otsu_threshold, image_result = cv2.threshold(gray_flat, 0, 255, cv2.THRESH_OTSU)
    histr = cv2.calcHist([image_result],[0],None,[256],[0,256])

    return otsu_threshold

In [2]:
import os
import cv2
import numpy as np

labels = ["tinggi", "sedang", "normal"]
direktori = "./Dataset/"
list_img = {}
result = []

for label in labels :
    dummy_images = []
    for count, data in enumerate(os.listdir(direktori + label)) :
        img = cv2.imread(direktori + label + "/" + data)
        result.append(label + "," + str(houghTransform(img)))
        dummy_images.append(data)
    list_img[label] = dummy_images

print(result)
f = open("result.csv", "w+")
f.write("label,value\n")
for r in result :
    f.write(r + "\n")
f.close()

    

['tinggi,106.0', 'tinggi,144.0', 'sedang,91.0', 'sedang,89.0', 'normal,58.0', 'normal,47.0']
